### GENERATING THE DATA TO TRAIN DE MODEL

Randomly produce "brush strokes" matrices with 3 + 1 dimensions:

* D1 Height
* D2 Width
* D3 RGB color codification 
* D4 Depth : which will codify the layer in the picture, from 0 to N_layers

Notes: H & W could be fixed to certain values although higher enough to be able to capture enough information later (check this out) 
 
Layers: When assigning layers, there is no need to increase the number of the layer if there is not overlap on a previous layer. The number of layer should be the max(previous layers overlaped by the actual "brush stroke") + 1.



(note: the input data could be also a section of any of the original items produced)


## 1- Generating the brush strokes data base

from pictures with many brush strokes, extract each stroke and save them in separated files.



## Funcion: input -> fichero del que extraer brushes, output -> fichero con contours y ficheros con brush strokes unitarios




In [3]:
import cv2
import numpy as np

# añadir la creación de todo el entramado de carpetas para generar los  datos desde 0



In [17]:
def extract_brushes_v2(path, nfile, nameInFile, nameOutContFile):
    
    image = cv2.imread(path + nameInFile)
    image = (255-image)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #to gray scale
    (thresh, blackAndWhiteImage) = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)

    scale_percent= 600/blackAndWhiteImage.shape[1] #percent of established width in order to have all inputs in same scale for a later application a minimum area

    width = int(blackAndWhiteImage.shape[1] * scale_percent )
    height = int(blackAndWhiteImage.shape[0] * scale_percent )

    # resize image
    blackAndWhiteImage = cv2.resize(blackAndWhiteImage, (width, height))
    image = cv2.resize(image, (width, height))
    image0 = image.copy()

    canny = cv2.Canny(blackAndWhiteImage, 10, 150)
    canny = cv2.dilate(canny, None, iterations=1)
    canny = cv2.erode(canny, None, iterations=1)  
    
  
    contours,hierarchy = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    threshold_area = 60  
    
    new_contours=[]    

    for i,cnt in enumerate(contours):        
        area = cv2.contourArea(cnt)         

        if area > threshold_area:             
            new_contours.append(cnt)
            cv2.drawContours(blackAndWhiteImage, contours,i, (0,255,0), 3)  # se marcan los contours en verde

    cv2.imwrite(path + nameOutContFile, image) # almacenamos en un fichero la salida para seguimiento-validación    
    
    
    # extraer cada pincelada por separado en un fichero
      
    
    return

In [14]:
def extract_brushes(path, nfile, nameInFile, nameOutContFile):
    
    image = cv2.imread(path + nameInFile)
    image = (255-image)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #to gray scale
    

    scale_percent= 800/gray.shape[1] #percent of established width in order to have all inputs in same scale for a later application a minimum area

    width = int(gray.shape[1] * scale_percent )
    height = int(gray.shape[0] * scale_percent )

    # resize image
    gray = cv2.resize(gray, (width, height))
    image = cv2.resize(image, (width, height))
    image0 = image.copy()

    canny = cv2.Canny(gray, 10, 150)
    canny = cv2.dilate(canny, None, iterations=1)
    canny = cv2.erode(canny, None, iterations=1)  
    
  
    contours,hierarchy = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    threshold_area = 100  
    
    new_contours=[]    

    for i,cnt in enumerate(contours):        
        area = cv2.contourArea(cnt)         

        if area > threshold_area:             
            new_contours.append(cnt)
            cv2.drawContours(image, contours,i, (0,255,0), 3)  # se marcan los contours en verde
            
    path_contours='./data/brush_strokes/01_contours'
    cv2.imwrite(path_contours + nameOutContFile, image) # almacenamos en un fichero la salida para seguimiento-validación    
    
    # extraer cada pincelada por separado en un fichero
    for idx in range(len(new_contours)):
        image_i=image0.copy() #actuamos sobre una copia
        mask_i = np.zeros_like(image_i) # Create mask where white is what we want, black otherwise
        cv2.drawContours(mask_i, new_contours, idx, (255,255,255), -1) # Draw filled contour in mask
        out_i = np.zeros_like(image_i)   # Extract out the object and place into output image
        out_i[mask_i == 255] = image_i[mask_i == 255]
        
        #crop out____________________________________________
        
        (x, y) = np.where((mask_i[:, :, 0]==255) & (mask_i[:, :, 1]==255) & (mask_i[:, :, 2]==255))

        (topx, topy) = (np.min(x), np.min(y))
        (bottomx, bottomy) = (np.max(x), np.max(y))
        
        height=bottomy-topy
        width=bottomx-topx

        brush = out_i[topx: topx + width,topy: topy + height]
        
        path_brushes='./data/brush_strokes/01_contours'
        cv2.imwrite(path_brushes + "bs_uniq_" + str(nfile) + "_" + str(idx)+ ".jpg", brush)
    
    
    
    return

In [19]:
# geneando la ddbb de brush strokes #
#-----------------------------------#

path_bs='./data/brush_strokes/'

for idfile in range(25): 
    file_n="0" + str(idfile + 1)
    file_n=file_n[(len(file_n)-2):len(file_n):]
    
    
    file_is =  "bs_{}.jpg".format(file_n)
    file_tobe =  "bs_{}_contours.jpg".format(file_n )
    extract_brushes_v2(path_bs,idfile, file_is,file_tobe)   
  

In [12]:
#Test individual
path_bs='./data/brush_strokes/'
extract_brushes(path_bs,1, "bs_01.jpg","bs_01_contours.jpg")

### Nota_  Se pueden incluir pinceladas desde fotografias. Puedo crear mi propia base de datos de pinceladas, pintándolas.

De estas páginas se pueden extraer brush strokes, utilizarlas para la v final

https://stock.adobe.com/es/search?filters%5Bcontent_type%3Aphoto%5D=1&filters%5Bcontent_type%3Aillustration%5D=1&filters%5Bcontent_type%3Azip_vector%5D=1&filters%5Bcontent_type%3Avideo%5D=1&filters%5Bcontent_type%3Atemplate%5D=1&filters%5Bcontent_type%3A3d%5D=1&filters%5Bcontent_type%3Aaudio%5D=0&filters%5Binclude_stock_enterprise%5D=0&filters%5Bis_editorial%5D=0&filters%5Bfree_collection%5D=0&order=relevance&serie_id=189411385&continue-checkout=1


https://www.vecteezy.com/vector-art/4246949-brush-strokes-bundle-vector-paintbrush-set-round-frames-set-of-strokes-grunge-brush-stroke



### NOTA:  si todos los orígenes de datos son del mismo tamaño, no habrá problemas con el formateo, de lo contrario, se encuentran parámetros que tienen sentido para unas fotos mas que otras!!

In [ ]:
#CODIGO INICIAL, SIN LIMPIAR_______________________________________________________
########################################################################
########################################################################
# de momento, el mejor código para hacer lo que quiero*
# LIMITANDO EL AREA DE LA SUPERFICIE PARA DEJAR DE LADO CONTOURS DESESTIMABLES
########################################################################

import cv2
import numpy as np


path_bs='./data/brush_strokes/'
image = cv2.imread(path_bs + "bs_01.jpg")
image = (255-image)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #to gray scale

scale_percent= 800/gray.shape[1] #percent of established width in order to have all inputs in same scale for a later application a minimum area

width = int(gray.shape[1] * scale_percent )
height = int(gray.shape[0] * scale_percent )
  
# resize image
gray = cv2.resize(gray, (width, height))
image = cv2.resize(image, (width, height))
image0 = image.copy()

canny = cv2.Canny(gray, 10, 150)
canny = cv2.dilate(canny, None, iterations=1)
canny = cv2.erode(canny, None, iterations=1)


contours,hierarchy = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

threshold_area = 100 

cont_areas=[]
cont_areas_thres=[]
new_contours=[]

for i,cnt in enumerate(contours):        
    area = cv2.contourArea(cnt) 
    cont_areas.append(area)
    
    if area > threshold_area: 
        cont_areas_thres.append(area)
        new_contours.append(cnt)
        cv2.drawContours(image, contours,i, (0,255,0), 3)  # se marcan los contours en verde
         
cv2.imwrite("result_Ok1_area.png", image) # almacenamos en un fichero la salida para seguimiento-validación

cv2.imshow('image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
############################################################################################################
#FINAL extracción del contour 1, generar un bucle desde hasta len(new_contours)
############################################################################################################
idx = 1

mask = np.zeros_like(image0) # Create mask where white is what we want, black otherwise
cv2.drawContours(mask, new_contours, idx, (255,255,255), -1) # Draw filled contour in mask
out = np.zeros_like(image0)   # Extract out the object and place into output image
out[mask == 255] = image0[mask == 255]
#grayImage = cv2.cvtColor(out, cv2.COLOR_BGR2GRAY)
#(thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 127, 255, cv2.THRESH_BINARY)

cv2.imwrite("result_Ok1_out.png", out)
cv2.imshow('out',out)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
#recortar el elemento y guardarlo
############################################################

(x, y) = np.where((mask[:, :, 0]==255) & (mask[:, :, 1]==255) & (mask[:, :, 2]==255))

(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
#topx, topy,bottomx,bottomy
height=bottomy-topy
width=bottomx-topx

brush = out[topx: topx + width,topy: topy + height]

cv2.imwrite("result_Ok1_roi.png", brush)
cv2.imshow('out',roi)
cv2.waitKey(0)
cv2.destroyAllWindows()
